<hr><center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks&para;&para;">unSkript Runbooks<a class="jp-InternalAnchorLink" href="#unSkript-Runbooks" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" rel="noopener" data-commandlinker-command="rendermime:handle-local-link" data-commandlinker-args="{&quot;path&quot;:&quot;102cd538-b28a-4b74-8893-861ce9824189/current/#unSkript-Runbooks&para;&quot;,&quot;id&quot;:&quot;#unSkript-Runbooks%C2%B6&quot;}">&para;</a><a class="jp-InternalAnchorLink" href="#unSkript-Runbooks&para;&para;" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success">
<h3 id="Objective&para;&para;">Objective<a class="jp-InternalAnchorLink" href="#Objective" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="../../../../../../files/102cd538-b28a-4b74-8893-861ce9824189/current/%23Objective%C2%B6?_xsrf=2%7C8f9a50d0%7C0b0c9812a06894c95a6f3bd724b42d0a%7C1676185969#Objective%C2%B6" target="_self" rel="noopener" data-commandlinker-command="rendermime:handle-local-link" data-commandlinker-args="{&quot;path&quot;:&quot;102cd538-b28a-4b74-8893-861ce9824189/current/#Objective&para;&quot;,&quot;id&quot;:&quot;#Objective%C2%B6&quot;}">&para;</a><a class="jp-InternalAnchorLink" href="#Objective&para;&para;" target="_self">&para;</a></h3>
<br><strong style="color: #000000;"><em>Stop untagged EC2 Instances</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Restart-Unhealthy-Services-in-Target-Group&para;&para;"><u>Restart Unhealthy Services in Target Group</u><a class="jp-InternalAnchorLink" href="#K8S-Pod-in-CrashLoopBack-State" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="../../../../../../files/102cd538-b28a-4b74-8893-861ce9824189/current/%23Stop-Untagged-EC2-Instances%C2%B6?_xsrf=2%7C8f9a50d0%7C0b0c9812a06894c95a6f3bd724b42d0a%7C1676185969#Stop-Untagged-EC2-Instances%C2%B6" target="_self" rel="noopener" data-commandlinker-command="rendermime:handle-local-link" data-commandlinker-args="{&quot;path&quot;:&quot;102cd538-b28a-4b74-8893-861ce9824189/current/#Stop-Untagged-EC2-Instances&para;&quot;,&quot;id&quot;:&quot;#Stop-Untagged-EC2-Instances%C2%B6&quot;}">&para;</a><a class="jp-InternalAnchorLink" href="#Restart-Unhealthy-Services-in-Target-Group&para;&para;" target="_self">&para;</a></h2>
</center>
<h1 id="Steps-Overview&para;&para;">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="../../../../../../files/102cd538-b28a-4b74-8893-861ce9824189/current/%23Steps-Overview%C2%B6?_xsrf=2%7C8f9a50d0%7C0b0c9812a06894c95a6f3bd724b42d0a%7C1676185969#Steps-Overview%C2%B6" target="_self" rel="noopener" data-commandlinker-command="rendermime:handle-local-link" data-commandlinker-args="{&quot;path&quot;:&quot;102cd538-b28a-4b74-8893-861ce9824189/current/#Steps-Overview&para;&quot;,&quot;id&quot;:&quot;#Steps-Overview%C2%B6&quot;}">&para;</a><a class="jp-InternalAnchorLink" href="#Steps-Overview&para;&para;" target="_self">&para;</a></h1>
<p>1)&nbsp;<a href="#1">List Unhealthy Instances in a Target Group</a><br>2)&nbsp;<a href="#2" target="_self" rel="noopener">Restart EC2 instances</a></p>
<hr>

<h3 id="List-all-AWS-Regions">List all AWS Regions<a class="jp-InternalAnchorLink" href="#List-all-AWS-Regions" target="_self">&para;</a></h3>
<p>This action fetches all AWS Regions to execute Step 1👇. This action will only execute if no <code>region</code> is provided.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>region</code></p>
</blockquote>

In [5]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_list_all_regions_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_all_regions(handle) -> List:
    """aws_list_all_regions lists all the AWS regions

        :type handle: object
        :param handle: Object returned from Task Validate

        :rtype: Result List of result
    """

    result = handle.aws_cli_command("aws ec2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text")
    if result is None or result.returncode != 0:
        print("Error while executing command : {}".format(result))
        return str()
    result_op = list(result.stdout.split("\n"))
    list_region = [x for x in result_op if x != '']
    return list_region


task = Task(Workflow())
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not region",
    "condition_result": true
    }''')

task.configure(outputName="region")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_all_regions, lego_printer=aws_list_all_regions_printer, hdl=hdl, args=args)

<h3 id="List-Unhealthy-Instances-in-a-Target-Group&para;"><a id="2" target="_self" rel="nofollow"></a>List Unhealthy Instances in a Target Group</h3>
<p>Here we will fetch all the untagged&nbsp; EC2 instances.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters: <code>region(Optional)</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>unheathy_instances</code></p>
</blockquote>

In [6]:
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from unskript.legos.utils import CheckOutput, CheckOutputStatus
from unskript.legos.utils import parseARN
from typing import List, Optional, Tuple
import pprint

def aws_list_unhealthy_instances_in_target_group_printer(output):
    if output is None:
        return
    if isinstance(output, CheckOutput):
        print(output.json())
    else:
        pprint.pprint(output)

def get_all_target_groups(handle, r):
    target_arns_list = []
    elbv2Client = handle.client('elbv2', region_name=r)
    try:
        tbs = aws_get_paginator(elbv2Client, "describe_target_groups", "TargetGroups")
        for index, tb in enumerate(tbs):
            target_arns_list.append(tb.get('TargetGroupArn'))
    except Exception as e:
        pass
    return target_arns_list

def aws_list_unhealthy_instances_in_target_group(handle, region: str=None) -> Tuple:
    result = []
    unhealthy_instances_list = []
    all_target_groups = []
    unhealhthy_instances_dict ={}
    all_regions = [region]
    if region is None or len(region)==0:
        all_regions = aws_list_all_regions(handle=handle)
    for r in all_regions:
        try:
            output = get_all_target_groups(handle,r)
            if len(output)!=0:
                all_target_groups.append(output)
        except Exception as e:
            pass
    for target_group in all_target_groups:
        for o in target_group:
            parsedArn = parseARN(o)
            region_name = parsedArn['region']
            elbv2Client = handle.client('elbv2', region_name=region_name)
            try:
                targetHealthResponse = elbv2Client.describe_target_health(TargetGroupArn=o)
            except Exception as e:
                raise e
            for ins in targetHealthResponse["TargetHealthDescriptions"]:
                if ins['TargetHealth']['State'] in ['unhealthy']:
                    unhealthy_instances_list.append(ins['Target']['Id'])
    if len(unhealthy_instances_list)!=0:
        unhealhthy_instances_dict['instance'] = unhealthy_instances_list
        unhealhthy_instances_dict['region'] = region_name
        result.append(unhealhthy_instances_dict)
    if len(result)!=0:
        return (False,result)
    else:
        return (True, None)

task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "region",
    "iter_parameter": "region"
    }''')
task.configure(outputName="unhealthy_instances")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_unhealthy_instances_in_target_group, lego_printer=aws_list_unhealthy_instances_in_target_group_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-untagged-instances&para;"><a id="2" target="_self" rel="nofollow"></a>Create List of unhealthy instances</h3>
<blockquote>
<p>This action captures the following output: <code>all_unhealthy_instances</code></p>
</blockquote>

In [7]:
all_unhealthy_instances =[]
for k,v in unhealthy_instances.items():
    for each_output in v:
        if type(each_output)== list:
            if len(each_output)!=0:
                for each_instance in each_output:
                    all_unhealthy_instances.append(each_instance)
print(all_unhealthy_instances)

<h3 id="List-Unhealthy-Instances-in-a-Target-Group&para;"><a id="2" target="_self" rel="nofollow"></a>Restart EC2 instances</h3>
<p>Here we will restart all the unhealthy EC2 instances.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters: <code>region, instance_ids</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>None</code></p>
</blockquote>

In [ ]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint
from beartype import beartype

from beartype import beartype
@beartype
def aws_restart_ec2_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
@beartype
def aws_restart_ec2_instances(handle, instance_ids: List, region: str) -> Dict:
    """aws_restart_instances Restarts instances.

        :type handle: object
        :param handle: Object returned by the task.validate(...) method.

        :type instance_ids: list
        :param instance_ids: List of instance ids.

        :type region: string
        :param region: Region for instance.

        :rtype: Dict with the restarted instances info.
    """

    ec2Client = handle.client('ec2', region_name=region)
    res = ec2Client.reboot_instances(InstanceIds=instance_ids)
    return res


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "instance_ids": "iter.get(\\"instance\\")",
    "region": "iter.get(\\"region\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_unhealthy_instances",
    "iter_parameter": ["instance_ids","region"]
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_restart_ec2_instances, lego_printer=aws_restart_ec2_instances_printer, hdl=hdl, args=args)

<h3 id="Conclusion&para;&para;">Conclusion</h3>
<p>In this Runbook, we were able to restart all unhealthy EC2 instances in a target group using unSkript's AWS actions. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>